In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai

In [2]:
from inference_auth_token import get_access_token
from langchain_openai import ChatOpenAI

access_token = get_access_token()
base_url="https://data-portal-dev.cels.anl.gov/resource_server/sophia/vllm/v1"
#model ='meta-llama/Meta-Llama-3.1-405B-Instruct'
model = 'meta-llama/Llama-3.3-70B-Instruct'
try:
    llm = ChatOpenAI(model=model, base_url=base_url, api_key=access_token)
    print(f"Successfully loaded model: {model} from {base_url}")

except Exception as e:
    print(f"Error with loading {model}")
    print(e)

Successfully loaded model: meta-llama/Llama-3.3-70B-Instruct from https://data-portal-dev.cels.anl.gov/resource_server/sophia/vllm/v1


In [3]:
# Using pre-built ReAct agent from LangGraph example: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
from typing import Literal
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")

# Define the tools and graph

tools = [get_weather]
weather_graph = create_react_agent(llm, tools=tools)

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
            
inputs = {"messages": [("user", "what is the weather in sf")],}
print_stream(weather_graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

what is the weather in sf
================================== Ai Message ==================================
Tool Calls:
  get_weather (chatcmpl-tool-3eb1c6dfe4f448e9a471cc95937e18cf)
 Call ID: chatcmpl-tool-3eb1c6dfe4f448e9a471cc95937e18cf
  Args:
    city: sf
================================= Tool Message =================================
Name: get_weather

It's always sunny in sf
================================== Ai Message ==================================

I'm just kidding, I don't have have access to real-time weather information. The function call I provided earlier is the correct format for getting the weather in San Francisco using the `get_weather` function.


In [4]:
%%capture --no-stderr
%pip install pubchempy

In [5]:
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

@tool
def molecule_name_to_smiles(name: str) -> str:
    """Convert a molecule name SMILES format.

    Args:
        name (str): molecule name.

    Returns:
        str: SMILES string.
    """
    import pubchempy
    return pubchempy.get_compounds(str(name), "name")[0].canonical_smiles

tools = [molecule_name_to_smiles]
chem_graph = create_react_agent(llm, tools=tools)

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
            
inputs = {"messages": [("user", "what is SMILES string of Carbon Dioxide?")]}
print_stream(chem_graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

what is SMILES string of Carbon Dioxide?
================================== Ai Message ==================================
Tool Calls:
  molecule_name_to_smiles (chatcmpl-tool-4e7c9455b29c4ebea7bc66946dee6317)
 Call ID: chatcmpl-tool-4e7c9455b29c4ebea7bc66946dee6317
  Args:
    name: Carbon Dioxide
================================= Tool Message =================================
Name: molecule_name_to_smiles

C(=O)=O
================================== Ai Message ==================================

The SMILES string for Carbon Dioxide is "C(=O)=O".


In [6]:
inputs = {"messages": [("user", "what is SMILES string of 1-ethyl-3-(1-naphthalen-2-ylethenylamino)thiourea?")]}
print_stream(chem_graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

what is SMILES string of 1-ethyl-3-(1-naphthalen-2-ylethenylamino)thiourea?
================================== Ai Message ==================================
Tool Calls:
  molecule_name_to_smiles (chatcmpl-tool-99e85a4a83164f14809ce1e2d4e1224b)
 Call ID: chatcmpl-tool-99e85a4a83164f14809ce1e2d4e1224b
  Args:
    name: 1-ethyl-3-(1-naphthalen-2-ylethenylamino)thiourea
================================= Tool Message =================================
Name: molecule_name_to_smiles

CCNC(=S)NNC(=C)C1=CC2=CC=CC=C2C=C1
================================== Ai Message ==================================

The SMILES string of 1-ethyl-3-(1-naphthalen-2-ylethenylamino)thiourea is CCNC(=S)NNC(=C)C1=CC2=CC=CC=C2C=C1.
